<a href="https://colab.research.google.com/github/naoufal-mft/Trading_Robot/blob/API/reddit_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install asyncpraw


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 5.1 MB/s eta 0:00:00


In [2]:
pip install pandas


In [3]:
pip install praw


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 5.6 MB/s eta 0:00:00


In [19]:
import asyncio
import asyncpraw
import nest_asyncio
import pandas as pd
import praw
import datetime

nest_asyncio.apply()

async def main():
    async with asyncpraw.Reddit(
        client_id="A5JdkIx6Fd_92LzdvaRhLg",
        client_secret="mHYkASS6LWg0EObYuy745JXQmU6s0w",
        username="ExternalCoyote9645",
        password="Elmehdi85?",
        user_agent="APITUTO"
    ) as reddit:
        subreddit = await reddit.subreddit("stocks+TSLA+AAPL+finance+investing+StockMarket+wallstreetbets+SecurityAnalysis")

        reddit_data = []
        comments_data = []

        async for submission in subreddit.new(limit=50):
            await submission.load()

            submission_comments = []

            async for comment in submission.comments:
                if isinstance(comment, praw.models.Comment):
                    if comment.score > 1:
                        comment_data = {
                            'Comment': comment.body,
                            'CommentScore': comment.score,
                            'SubmissionTitle': submission.title
                        }
                        comments_data.append(comment_data)

            date_created = datetime.datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')

            if submission.score > 1:
                reddit_data.append({
                    'Title': submission.title,
                    'Author': submission.author.name,
                    'Score': submission.score,
                    'Upvotes': submission.ups,
                    'Downvotes': submission.downs,
                    'URL': submission.url,
                    'DateCreated': date_created
                })

        # Enregistrez les données Reddit dans un fichier CSV
        df = pd.DataFrame(reddit_data)
        df.to_csv('reddit_data.csv', index=False)

        # Enregistrez les commentaires dans un fichier CSV distinct
        df_comments = pd.DataFrame(comments_data)
        df_comments.to_csv('comments_data.csv', index=False)

        print(f'Données Reddit enregistrées dans reddit_data.csv')
        print(f'Commentaires enregistrés dans comments_data.csv')

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


Données Reddit enregistrées dans reddit_data.csv
Commentaires enregistrés dans comments_data.csv


In [6]:
import asyncio
import asyncpraw
import nest_asyncio
import pandas as pd
import praw
import datetime

nest_asyncio.apply()

async def main():
    # Créez une instance de Reddit en utilisant vos informations d'application
    reddit = asyncpraw.Reddit(
        client_id="A5JdkIx6Fd_92LzdvaRhLg",
        client_secret="mHYkASS6LWg0EObYuy745JXQmU6s0w",
        username="ExternalCoyote9645",
        password="Elmehdi85?",
        user_agent="APITUTO"
    )

    # Accédez à une subreddit spécifique (par exemple, r/stocks)
    subreddit = await reddit.subreddit("stocks+TSLA+AAPL+finance+investing+StockMarket+wallstreetbets+SecurityAnalysis")

    # Créez des listes pour stocker les données "good news" et les commentaires correspondants
    good_news_data = []
    good_news_comments = []

    # Créez des listes pour stocker les données "bad news" et les commentaires correspondants
    bad_news_data = []
    bad_news_comments = []

    # Parcourez les publications
    async for submission in subreddit.new(limit=70):
        # Chargez la soumission pour accéder à ses commentaires
        await submission.load()

        submission_comments = []

        # Parcourez les commentaires de la soumission
        async for comment in submission.comments:
            if isinstance(comment, praw.models.Comment):
                if comment.score > 10:
                    submission_comments.append({
                        'Comment': comment.body,
                        'CommentScore': comment.score,
                    })

        date_created = datetime.datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')

        # Examinez le contenu de la soumission pour déterminer s'il s'agit de "good news" ou "bad news"
        content = submission.title

        if "good" in content.lower() or "positive" in content.lower():
            # C'est une "good news", ajoutez-la à la liste correspondante
            good_news_data.append({
                'Title': submission.title,
                'Author': submission.author.name,
                'Score': submission.score,
                'Upvotes': submission.ups,
                'Downvotes': submission.downs,
                'URL': submission.url,
                'DateCreated': date_created
            })
            # Ajoutez les commentaires à la liste des commentaires "good news"
            good_news_comments.extend(submission_comments)
        elif "bad" in content.lower() or "negative" in content.lower():
            # C'est une "bad news", ajoutez-la à la liste correspondante
            bad_news_data.append({
                'Title': submission.title,
                'Author': submission.author.name,
                'Score': submission.score,
                'Upvotes': submission.ups,
                'Downvotes': submission.downs,
                'URL': submission.url,
                'DateCreated': date_created
            })
            # Ajoutez les commentaires à la liste des commentaires "bad news"
            bad_news_comments.extend(submission_comments)

    # Créez des DataFrames distincts pour les "good news" et les "bad news" ainsi que pour les commentaires correspondants
    df_good_news = pd.DataFrame(good_news_data)
    df_good_comments = pd.DataFrame(good_news_comments)
    df_bad_news = pd.DataFrame(bad_news_data)
    df_bad_comments = pd.DataFrame(bad_news_comments)

    # Enregistrez les DataFrames dans des fichiers CSV distincts
    csv_good_news_path = 'good_news_data.csv'
    csv_good_comments_path = 'good_comments_data.csv'
    csv_bad_news_path = 'bad_news_data.csv'
    csv_bad_comments_path = 'bad_comments_data.csv'

    df_good_news.to_csv(csv_good_news_path, index=False)
    df_good_comments.to_csv(csv_good_comments_path, index=False)
    df_bad_news.to_csv(csv_bad_news_path, index=False)
    df_bad_comments.to_csv(csv_bad_comments_path, index=False)

    print(f'Données "good news" enregistrées dans {csv_good_news_path}')
    print(f'Commentaires correspondants aux "good news" enregistrés dans {csv_good_comments_path}')
    print(f'Données "bad news" enregistrées dans {csv_bad_news_path}')
    print(f'Commentaires correspondants aux "bad news" enregistrés dans {csv_bad_comments_path}')

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


Données "good news" enregistrées dans good_news_data.csv
Commentaires correspondants aux "good news" enregistrés dans good_comments_data.csv
Données "bad news" enregistrées dans bad_news_data.csv
Commentaires correspondants aux "bad news" enregistrés dans bad_comments_data.csv
